In [1]:
import pandas as pd
import datetime
import numpy as np
from sklearn.model_selection import KFold
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
import warnings
warnings.filterwarnings("ignore") 
from matplotlib import pyplot as plt
from sklearn.metrics import f1_score
from scipy import stats
from sklearn.metrics import precision_score

In [2]:
Filepath = 'C:/Users/ChowdhKa/Downloads/Kakoli Masters/MRP/All File/'

In [3]:
# Read cluster file
fields = ['Channel', 'clusterNum']
clusterFile = pd.read_csv(Filepath+'All_Cluster.csv', skipinitialspace=True, usecols=fields)
clusterFile = clusterFile[(clusterFile['clusterNum'] == 1)| (clusterFile['clusterNum'] == 4)]

In [4]:
clusterFile.head()

,Channel,clusterNum
0,2340,1
1,10670,1
2,10930,1
3,11710,1
4,11720,1


In [5]:
clusterFile.shape

(96, 2)

In [6]:
#read layer 3 data csv
df=pd.read_csv(Filepath+'OctDataSmall.csv')
df.columns = ['StartDate', 'EndDate', 'Channel', 'Pwr_dbm','Snr', 'Occupancy']
df['StartDate'] = pd.to_datetime(df['StartDate'])
df['EndDate'] = pd.to_datetime(df['EndDate'])
df['StartDate'] = df['StartDate'].values.astype('<M8[h]')
df['EndDate'] = df['EndDate'].values.astype('<M8[h]')
df.head()

,StartDate,EndDate,Channel,Pwr_dbm,Snr,Occupancy
0,2016-10-06,2016-10-06 01:00:00,56303,-83.293709,21.397917,100.0
1,2016-10-06,2016-10-06 01:00:00,56304,-63.193748,41.497952,100.0
2,2016-10-06,2016-10-06 01:00:00,56305,-31.322813,73.445999,100.0
3,2016-10-06,2016-10-06 01:00:00,56306,-64.096245,41.568588,100.0
4,2016-10-06,2016-10-06 01:00:00,56307,-83.440269,21.249647,100.0


In [7]:
#putting dataframe in another dataframe 
dataf = df

In [8]:
# Take the data frame from R file
df_merged=pd.read_csv(Filepath+'testcsv.csv',delimiter="\t")
#print(df_merged)
df_merged.columns = ['Channel','StartDate', 'EndDate', 'Pwr_dbm','Snr', 'Occupancy','per_read']
df_merged['StartDate'] = pd.to_datetime(df_merged['StartDate'])
df_merged['EndDate'] = pd.to_datetime(df_merged['EndDate'])
df_merged['StartDate'] = df_merged['StartDate'].values.astype('<M8[h]')
df_merged['EndDate'] = df_merged['EndDate'].values.astype('<M8[h]')
df_merged.head()

,Channel,StartDate,EndDate,Pwr_dbm,Snr,Occupancy,per_read
1,10,2016-10-07 08:00:00,2016-10-07 09:00:00,-91.500000,11.700000,0.074493,82.82504
2,10,2016-10-21 21:00:00,2016-10-21 22:00:00,-90.774193,12.806452,0.466165,82.82504
3,10,2016-10-11 01:00:00,2016-10-11 02:00:00,-91.592590,11.444445,0.203912,82.82504
4,10,2016-10-21 12:00:00,2016-10-21 13:00:00,-91.340424,11.851064,0.352800,82.82504
5,10,2016-10-20 23:00:00,2016-10-21 00:00:00,-92.064514,11.580646,0.237639,82.82504


In [9]:
# Taking only the required columns
df_merged = df_merged[['Channel','StartDate', 'EndDate', 'Occupancy']]

In [10]:
df_merged.head(2)

,Channel,StartDate,EndDate,Occupancy
1,10,2016-10-07 08:00:00,2016-10-07 09:00:00,0.074493
2,10,2016-10-21 21:00:00,2016-10-21 22:00:00,0.466165


In [11]:
df_merged.shape

(1214664, 4)

In [12]:
clusterFile.head(2)

,Channel,clusterNum
0,2340,1
1,10670,1


In [13]:
# JOIN df_merged and clusterFile on channels
clusterChannelDF = pd.merge(clusterFile, df_merged[['Channel', 'StartDate', 'EndDate', 'Occupancy']],on='Channel')
clusterChannelDF = clusterChannelDF.drop(['clusterNum'], 1)

In [14]:
clusterChannelDF.head()

,Channel,StartDate,EndDate,Occupancy
0,2340,2016-10-15 00:00:00,2016-10-15 01:00:00,67.118799
1,2340,2016-10-16 00:00:00,2016-10-16 01:00:00,73.872579
2,2340,2016-10-12 06:00:00,2016-10-12 07:00:00,100.000000
3,2340,2016-10-28 17:00:00,2016-10-28 18:00:00,12.049958
4,2340,2016-10-31 09:00:00,2016-10-31 10:00:00,87.301233


In [15]:
clusterChannelDF.shape

(49536, 4)

In [16]:
#Getting the final channel list

channelList = clusterChannelDF.Channel.unique().tolist()
#channelList = np.random.choice(channel, 300, replace=False)

In [17]:
#channelList.tolist()

In [18]:
#channelList = [20750,22320,23200,28430,34680,34690,50267,15220,15530,15570,15820,16010,16130]#,
               #21340,24220,25410,53970,54006,54494,54620,56097,56293,56305,56306,56307]

In [19]:
#channelList = [20750,22320,23200,28430]
               #,34680,34690,50267,15220,15530,15570,15820,16010,16130,
               #21340,24220,25410,53970,54006,54494,54620,56097,56293,56305,56306,56307]

In [20]:
OriginalMergedDF = clusterChannelDF

In [21]:
def ProcessDF(channel, threshold):
    InterDF = OriginalMergedDF[(OriginalMergedDF.Channel == channel)][['StartDate', 'EndDate', 'Channel', 'Occupancy']]
    InterDF = InterDF.set_index('StartDate')
    InterDF = InterDF.sort_index()
    indexStart = InterDF.index[0]
    indexEnd = InterDF.index[-1]
    InterDF= InterDF.reindex(pd.date_range(start=indexStart, end=indexEnd, freq='1h'))
    shiftVales = 2
    InterDF['Occupancy1'] = InterDF['Occupancy'].shift()
    while shiftVales <=6:
        InterDF['Occupancy'+str(shiftVales)] = InterDF['Occupancy'+str(shiftVales-1)].shift()
        shiftVales=shiftVales+1
    # Convert index back to dataframe column
    InterDF.reset_index(level=0, inplace=True)
    InterDF.drop(InterDF.columns[[0,1]], axis=1, inplace=True)
    InterDF = InterDF.dropna(how='any', axis = 0)
    #InterDF['OccupancyLabel'] = np.where(InterDF['Occupancy'] > threshold, 1,0)
    InterDF['OccupancyLabel'] = pd.cut(InterDF['Occupancy'], [-np.inf, threshold, np.inf], labels=[0, 1])
    return InterDF

In [22]:
def GausNB(X_train, y_train, X_test, y_test):
    model = GaussianNB()
    model.fit(X_train, y_train)
    predicted = model.predict(X_test)
    confusion_NB = confusion_matrix(y_test, predicted, labels=[0,1])
    # Accuracy
    scoreNB = accuracy_score(y_test, predicted)
    # Sensitivity
    senseNB = confusion_NB[1,1]/(confusion_NB[1,0]+confusion_NB[1,1])
    # Specificity
    specificNB = confusion_NB[0,0]/(confusion_NB[0,0]+confusion_NB[0,1])
    # F-measure
    fmeasureNB = f1_score(y_test, predicted,average='weighted')
    # Precision
    precisionscoreNB = precision_score(y_test, predicted, average='weighted')

    return scoreNB, senseNB, specificNB, fmeasureNB, precisionscoreNB

In [23]:
def LogReg(X_train, y_train, X_test, y_test):
    reg = LogisticRegression()
    modfit = reg.fit(X_train, y_train)
    yPred = modfit.predict(X_test)
    confusion_LR = confusion_matrix(y_test, yPred, labels=[0,1])
    # Accuracy
    # scoreLR = modfit.score(X_test, y_test)
    scoreLR = (confusion_LR[0,0]+confusion_LR[1,1])/(confusion_LR[0,0]+confusion_LR[0,1]+confusion_LR[1,0]+confusion_LR[1,1])
    # Sensitivity
    senseLR = confusion_LR[1,1]/(confusion_LR[1,0]+confusion_LR[1,1])
    # Specificity
    specificLR = confusion_LR[0,0]/(confusion_LR[0,0]+confusion_LR[0,1])
    # F-measure
    fmeasureLR = f1_score(y_test, yPred, average='weighted')
    # Precision
    precisionscoreLR = precision_score(y_test, yPred, average='weighted')

    return scoreLR, senseLR, specificLR, fmeasureLR, precisionscoreLR

In [24]:
#print(k[1])

In [25]:
# Defining the threshold for class labeling
SaveDataDF = pd.DataFrame([])
t = 70
#list_threshold = [0, 10, 20, 30, 40, 50, 60, 70, 80, 90, 100]
#list_threshold = [40, 50, 60, 70]
#for t in list_threshold:
for channel in channelList:
    #print('Value of Threshold: ' + str(t))
    #print('Current channel is: '+ str(channel))

    ProcessedDF = ProcessDF(channel, t)
    ProcessedDFx = ProcessedDF[(ProcessedDF.Channel == channel)][['Channel', 'Occupancy1', 'Occupancy2', 'Occupancy3', 'Occupancy4', 'Occupancy5', 'Occupancy6']]
    ProcessedDFy = ProcessedDF[(ProcessedDF.Channel == channel)][['OccupancyLabel']] 
    #print(ProcessedDFy.OccupancyLabel.unique())
    #print(ProcessedDFy['OccupancyLabel'].value_counts())
    k = ProcessedDFy.groupby('OccupancyLabel').size()
    #print('k[0]: '+ str(k[0]))
    #print('k[1]: '+ str(k[1]))
    if (k[0] > 5 and k[1] > 5):
        RunIter = 1
        while RunIter <= 10:
            foldCounter = 1
            kc = KFold(n_splits = 10, shuffle=True)
            for train_x, test_x in kc.split(ProcessedDFx, ProcessedDFy):
                X_trainDF, X_testDF = ProcessedDFx.iloc[train_x], ProcessedDFx.iloc[test_x]
                y_trainDF, y_testDF = ProcessedDFy.iloc[train_x], ProcessedDFy.iloc[test_x]


                # Naive Bayes
                scoreNB, sensitivityNB, specificityNB, fmeasureNB, precisionNB = GausNB(X_trainDF, y_trainDF, X_testDF, y_testDF)
                #if len(y_trainDF.OccupancyLabel.unique()) > 1:
                # Logistic Regression
                scoreLR, sensitivityLR, specificityLR, fmeasureLR, precisionLR = LogReg(X_trainDF, y_trainDF, X_testDF, y_testDF)

                # PRINT ROW VALUE IN DATAFRAME- KEEP APPENDING
                SaveDataDF = SaveDataDF.append(pd.DataFrame({'Channel': channel, 'Threshold': t,
                                                             'RunCount': RunIter,'FoldCount': foldCounter,
                                                             'AccuracyLR': scoreLR, 'SensitivityLR': sensitivityLR,
                                                             'SpecificityLR': specificityLR, 'AccuracyNB': scoreNB,
                                                             'SensitivityNB': sensitivityNB, 'SpecificityNB': specificityNB,
                                                             'fmeasureNB': fmeasureNB, 'fmeasureLR': fmeasureLR,
                                                             'precisionLR': precisionLR, 'precisionNB': precisionNB
                                                            }, 
                                                            index=[0]), ignore_index=True)
            foldCounter = foldCounter + 1
            RunIter = RunIter + 1


In [26]:
#fmeasureLR

In [27]:
#ProcessedDFy.head()

In [28]:
ProcessedDFx.head()

,Channel,Occupancy1,Occupancy2,Occupancy3,Occupancy4,Occupancy5,Occupancy6
6,56381.0,5.999848,16.680492,19.736352,36.970781,50.900834,62.322293
7,56381.0,10.429859,5.999848,16.680492,19.736352,36.970781,50.900834
8,56381.0,12.751678,10.429859,5.999848,16.680492,19.736352,36.970781
9,56381.0,7.082002,12.751678,10.429859,5.999848,16.680492,19.736352
10,56381.0,21.948991,7.082002,12.751678,10.429859,5.999848,16.680492


In [29]:
SaveDataDF.head()

,AccuracyLR,AccuracyNB,Channel,FoldCount,RunCount,SensitivityLR,SensitivityNB,SpecificityLR,SpecificityNB,Threshold,fmeasureLR,fmeasureNB,precisionLR,precisionNB
0,0.897959,0.836735,2340,1,1,0.800000,0.733333,0.941176,0.882353,70,0.896939,0.836735,0.896793,0.836735
1,0.775510,0.795918,2340,1,1,0.625000,0.750000,0.848485,0.818182,70,0.773565,0.798732,0.772309,0.804257
2,0.857143,0.816327,2340,1,1,0.600000,0.500000,0.923077,0.897436,70,0.854288,0.812656,0.852381,0.809807
3,0.918367,0.897959,2340,1,1,0.666667,0.777778,0.975000,0.925000,70,0.914261,0.899999,0.915452,0.903035
4,0.857143,0.836735,2340,1,1,0.800000,0.800000,0.882353,0.852941,70,0.858381,0.839363,0.860390,0.844913


In [30]:
SaveDataDF.to_csv(Filepath+'OutputData.csv', header=False,sep='\t', encoding='utf-8', index=False)

In [31]:
readResults = pd.read_csv(Filepath+'OutputData.csv', header=1, sep ='\t')

In [32]:
readResults.columns = ['AccuracyLR', 'AccuracyNB', 'Channel', 'FoldCount', 'RunCount', 
                    'SensitivityLR', 'SensitivityNB','SpecificityLR', 
                    'SpecificityNB','Threshold','FmeasureLR', 'FmeasureNB', 'PrecisionLR', 'PrecisionNB']

In [33]:
readResults.head()

,AccuracyLR,AccuracyNB,Channel,FoldCount,RunCount,SensitivityLR,SensitivityNB,SpecificityLR,SpecificityNB,Threshold,FmeasureLR,FmeasureNB,PrecisionLR,PrecisionNB
0,0.857143,0.816327,2340,1,1,0.600000,0.500000,0.923077,0.897436,70,0.854288,0.812656,0.852381,0.809807
1,0.918367,0.897959,2340,1,1,0.666667,0.777778,0.975000,0.925000,70,0.914261,0.899999,0.915452,0.903035
2,0.857143,0.836735,2340,1,1,0.800000,0.800000,0.882353,0.852941,70,0.858381,0.839363,0.860390,0.844913
3,0.877551,0.816327,2340,1,1,0.800000,0.700000,0.897436,0.846154,70,0.881506,0.824632,0.888950,0.839482
4,0.916667,0.812500,2340,1,1,0.937500,0.812500,0.906250,0.812500,70,0.917774,0.815925,0.922222,0.825771


In [34]:
pVal = stats.ttest_ind(SaveDataDF['AccuracyNB'][:800],SaveDataDF['AccuracyLR'][:800], nan_policy = 'propagate')
pValue = pVal[1]

In [35]:
pVal

Ttest_indResult(statistic=-15.761794348344937, pvalue=3.899707912671742e-52)

In [36]:
readResults = readResults.drop(['FoldCount', 'RunCount'], 1)

In [37]:
PerThresPerChannelDF = readResults.groupby(['Channel','Threshold']).mean().add_suffix('_mean').reset_index()

In [38]:
PerThresPerChannelDF.head()

,Channel,Threshold,AccuracyLR_mean,AccuracyNB_mean,SensitivityLR_mean,SensitivityNB_mean,SpecificityLR_mean,SpecificityNB_mean,FmeasureLR_mean,FmeasureNB_mean,PrecisionLR_mean,PrecisionNB_mean
0,2340,70,0.886037,0.844448,0.797709,0.790770,0.916741,0.863543,0.886504,0.848347,0.892380,0.861517
1,10670,70,0.954715,0.920561,0.966395,0.899076,0.943233,0.944176,0.954695,0.920557,0.956814,0.924691
2,10930,70,0.954145,0.919626,0.954563,0.911890,0.954568,0.927681,0.954173,0.919657,0.955748,0.922636
3,11710,70,0.953108,0.912360,0.959656,0.907964,0.946894,0.917566,0.953119,0.912349,0.954853,0.915905
4,11720,70,0.950629,0.923852,0.964034,0.928103,0.938056,0.920403,0.950649,0.923830,0.952739,0.927676


In [39]:
PerThresPerChannelDF['CountChan'] = 1
aggregator = {'AccuracyLR_mean' : 'mean',
              'AccuracyNB_mean' : 'mean',
              'SensitivityLR_mean': 'mean',
              'SensitivityNB_mean' : 'mean',
              'SpecificityLR_mean': 'mean',
              'SpecificityNB_mean' : 'mean',
              'FmeasureLR_mean':'mean',
              'FmeasureNB_mean':'mean',
              'PrecisionLR_mean': 'mean',
              'PrecisionNB_mean': 'mean',
              'CountChan': 'sum'}
FinalResultsDF = PerThresPerChannelDF.groupby('Threshold', as_index=False)['AccuracyLR_mean', 'AccuracyNB_mean', 
                                                                 'SensitivityLR_mean', 'SensitivityNB_mean', 
                                                                 'SpecificityLR_mean', 'SpecificityNB_mean',
                                                                 'PrecisionLR_mean','PrecisionNB_mean',
                                                                 'FmeasureLR_mean', 'FmeasureNB_mean'].agg(aggregator)


In [40]:
#PerThresPerChannelDF = PerThresPerChannelDF.drop(['Channel'], 1)
#FinalResultsDF = PerThresPerChannelDF.groupby(['Threshold']).mean().reset_index()
FinalResultsDF['Cluster'] = '14'
FinalResultsDF['pValue'] = pValue

In [41]:
FinalResultsDF

,Threshold,AccuracyLR_mean,AccuracyNB_mean,SensitivityLR_mean,SensitivityNB_mean,SpecificityLR_mean,SpecificityNB_mean,FmeasureLR_mean,FmeasureNB_mean,PrecisionLR_mean,PrecisionNB_mean,CountChan,Cluster,pValue
0,70,0.939533,0.901293,0.910757,0.891423,0.940299,0.904008,0.939007,0.903074,0.942067,0.91132,96,14,3.899708e-52


In [42]:
# Write the results to a CSV
FinalResultsDF.to_csv(Filepath+'Output/FinalResults.csv', header=False,sep='\t', index=False, mode = 'a')